In [ ]:
!pip install openai

In [ ]:
!pip install mysql-connector-python

In [ ]:
# package

import openai
import mysql.connector
from pprint import pprint # chatGPT message가 잘 출력되되도록 사용
import pandas as pd
import numpy as np

In [ ]:
# Google Drive 마운트
# 마운트는 본인 드라이브만 가능하기에 공유 폴더 같은 경우 본인 드라이브에 바로가기를 추가하여 사용

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# API KEY : contact 계정의 API key 발급하여 사용

openai.api_key = ""

In [ ]:
# google sheet를 바로 연동하는 것은 원본 훼손 위험이 존재하기에 우선 xlsx 파일을 다운로드 및 업로드하여여 활용

raw_data = pd.read_excel("/content/drive/MyDrive/HAI 연구개발 리서치/data/PoC 평가.xlsx", sheet_name='total')

In [ ]:
en_data = raw_data.copy()

In [ ]:
# PoC 평가가 A+인 인원들만 불러오기
# url 주소에 userID가 존재하는 형식 데이터만 불러오기
en_data = en_data.loc[en_data['총평 \n(Letter grade, like A+, B+)'] == 'A+']
en_data = en_data[en_data['framing docs'].str.contains('&userID', na = False)].reset_index(drop=True)

In [ ]:
# '&' 기준으로 파싱하여 user_id 리스트에 저장하기
id_list = []
for i in range(len(en_data)) :
  id_list.append(en_data['framing docs'][i].split('&')[1][7:])

In [ ]:
# DB 연동

mydb = mysql.connector.connect(
  host="",
  user="",
  password="",
  database=""
)

# 저장된 userID 정보 불러와서 사용
userCode = id_list[1]

mycursor = mydb.cursor()

mycursor.execute("SELECT user_id from userCodeTable where code='" + userCode + "'")

myresult = mycursor.fetchall()

userID = myresult[0][0]

mycursor.execute("SELECT user_id, question_id, response FROM responseTable where user_id='" + str(userID) + "'")

myresult = mycursor.fetchall()

userResponse = {}

# 0부터 시작되는 문항번호를 1부터 시작하도록 변경
for x in myresult:
    userResponse[x[1]+1] = x[2]

In [ ]:
# PoC 데이터는 dictionary 형태로 저장
userResponse

In [ ]:
# 문항 정보 입력
qna = pd.read_csv("/content/drive/MyDrive/HAI 연구개발 리서치/data/PoC_Question.csv")

In [ ]:
# 초기 system role 세팅


In [ ]:
i = 22
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
            {"role" : "system", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"}
            ]

messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : '원재료1-유동지수/원재료1-충격강도/원재료2-유동지수/원재료2-녹는점/원재료3-인열강도/ ... 원재료n-상대점도/제품의 유동지수'"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
            {"role" : "system", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"},
            {"role" : "user", "content" : f"{i}번 문항 답변 : '원재료1-유동지수/원재료1-충격강도/원재료2-유동지수/원재료2-녹는점/원재료3-인열강도/ ... 원재료n-상대점도/제품의 유동지수'"},
            "role" : "assistant", "content" : chat_response}
            ]

In [ ]:
# 6개 문항 선택
i = 5
messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse[i]}"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
i = 8
messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse[i]}"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
i = 22
messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse[i]}"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
messages.append({"role" : "user", "content" : "위 22번에 대한 예시로 10개 row를 보유하고 있는 데이터프레임을 구성해줘"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
messages.append({"role" : "user", "content" : "위 구성한 데이터프레임을 활용하여 인공지능 모델 구축 코드를 만들고 해당 metrics까지 산출해줘"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
# 초기 system role 세팅
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"}]

In [ ]:
i = 22
messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse[i]}"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
messages.append({"role" : "user", "content" : "위 22번에 대한 예시로 10개 row를 보유하고 있는 데이터프레임을 구성해줘"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
messages.append({"role" : "user", "content" : "위 데이터프레임을 feature 명칭, Label 명칭을 참고하여 python code로 생성한 후, csv 파일로 결과를 얻을 수 있도록 코드를 구축해줘"})

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=1
)

chat_response = completion.choices[0].message.content
messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
messages

In [ ]:
# 일단 대충 얼기설기 짜놓았습니다.
# 앞에서부터 탐색 / 뒤에서부터 탐색 두개로 각각 발견되면 멈추게 짜면 될 것 같습니다.
divided = chat_response.split('\n')
idx = []
for i, w in enumerate(divided):
    if w.startswith("```") or w.endswith("```"):
        idx.append(i)
output_code = '\n'.join(divided[idx[0]+1:idx[-1]])

pprint(output_code)

In [ ]:
with open('/content/drive/MyDrive/HAI 연구개발 리서치/output/output_script_2.py', 'w') as f:
    f.write(output_code)

In [ ]:
def Code_CoT_test(q_1:str, q_2:str, q_3:str, plot:list, ChatGPT_api_key:str, data_path:str, model:str="gpt-3.5-turbo")->str:

    openai.api_key = ChatGPT_api_key

    messages = [
        {
            "role": "system",
            "content": "인공지능 모델을 활용하여 개선이 가능한 영역인지 확인해줘",
        },

        {
            "role": "system",
            "content": "인공지능을 활용했을 때, 회귀, 분류 모델 중 어떤 모델이 적합한지 확인해줘",
        },

        {
            "role": "system",
            "content": "개선이 필요한 영역을 한 줄로 정리해서 요약해줘",
        },

        {
            "role": "user",
            "content": f'인공지능을 적용하여여 개선이 필요한 영역은 "{q_1}"이야'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)

    answer_1 = response["choices"][0]["message"]["content"]

    messages = [
        {
            "role": "system",
            "content": f"'{answer_1}' 영역에 대해 인공지능을 활용하고 싶어",
        },

        {
            "role": "system",
            "content": "인공지능이 예측하려는 Target이 무엇인지 확인해줘",
        },

        {
            "role": "system",
            "content": "인공지능의 목적을 한 줄로 정리해서 요약해줘.",
        },

        {
            "role": "user",
            "content": f'인공지능의 목적은 "{q_2}"이야'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_2 = response["choices"][0]["message"]["content"]

    messages = [
        {
            "role": "system",
            "content": f"AI의 주제는 '{answer_2}'이야",
        },
        {
            "role": "system",
            "content": "입력된 데이터를 feature와 label 두 종류로 구분해줘",
        },

        {
            "role": "system",
            "content":"python script를 사용하여 dataframe을 생성해줘",
        },

        {
            "role": "system",
            "content": "결과 전달 시 python script만 출력해줘",
        },

        {
            "role": "system",
            "content": "dataframe 생성 시, column의 명칭은 입력된 feature와 Label 명칭으로 사용해줘",
        },

        {
            "role": "system",
            "content": "dataframe의 value값들은 임의로 10개만 생성해줘",
        },

         {
            "role": "system",
            "content": f"생성한 dataframe의 저장 경로는 {data_path}야",
        },

         {
            "role": "system",
            "content": "위 단계들을 차근차근 생각해줘",
        },
        {
            "role": "user",
            "content": f"예시 데이터 세트는 '{q_3}'이야. dataframe을 생성하고 저장해줘"
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_3 = response["choices"][0]["message"]["content"]

    messages = [
            {
                "role": "system",
                "content": f"인공지능의 목적은 '{answer_2}'이야",
            },
            {
                "role": "system",
                "content": f"현재 보유한 데이터프레임은 {answer_3}이야",
            },

            {
                "role": "system",
                "content": "AI Modeling 관점으로 사용가능한 plot들을 1개만 추천해줘",
            },

            {
                "role": "system",
                "content":"데이터프레임의 feature들을 참고하여 사용가능한 plot들을 1개만 추천해줘",
            },

            {
                "role": "system",
                "content": f"plot들을 추천할 시 {plot} 리스트 목록에 존재하는 plot들만 추천해줘",
            },

            {
                "role": "user",
                "content": '데이터프레임을 참고하여 필요한 시각화 plot들을 추천해줘'
            }
        ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_4 = response["choices"][0]["message"]["content"]

    return answer_3, answer_4
'''
    messages = [
        {
            "role": "system",
            "content": f"지금부터 {answer_3}를 리뷰할거야",
        },
        {
            "role": "system",
            "content": "차근차근 생각하고 효율적으로 코드를 구축해줘",
        },

        {
            "role": "system",
            "content": "결과 전달 시 python script만 출력해줘",
        },

        {
            "role": "system",
            "content":f"생성한 {answer_3}가 실행하는데 문제가 없는지 체크해줘",
        },

        {
            "role": "system",
            "content": f"실행하는데 문제가 발생하지 않을 경우 {answer_3}를 그대로 둬",
        },

        {
            "role": "user",
            "content": f'생성한 {answer_3}를 리뷰해주고 python script를 다시 전달해줘'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer_3 = response["choices"][0]["message"]["content"]
'''

In [ ]:
output, output_2 = Code_CoT_test(q_1=userResponse[5], q_2=userResponse[8], q_3=userResponse[22],
                      plot = ["feature_importance", "roc_curve", "bar_plot", "line_plot", "pie_plot", "heatmap", "confusion_matrix", "histogram"],
                      ChatGPT_api_key=openai.api_key,
                      data_path='/content/drive/MyDrive/HAI 연구개발 리서치/output/',
                      model="gpt-3.5-turbo")

In [ ]:
from pprint import pprint

In [ ]:
pprint(output)

In [ ]:
print(output_2)

In [ ]:
# 일단 대충 얼기설기 짜놓았습니다.
# 앞에서부터 탐색 / 뒤에서부터 탐색 두개로 각각 발견되면 멈추게 짜면 될 것 같습니다.
divided = output.split('\n')
idx = []
for i, w in enumerate(divided):
    if w.startswith("```") or w.endswith("```"):
        idx.append(i)
output_code = '\n'.join(divided[idx[0]+1:idx[1]])

pprint(output_code)

In [ ]:
with open('/content/drive/MyDrive/HAI 연구개발 리서치/output/output_poc_3.py', 'w') as f:
    f.write(output_code)

In [ ]:
%run -i '/content/drive/MyDrive/HAI 연구개발 리서치/output/output_poc_2.py'

In [ ]:
"system : 인공지능 모델을 활용하여 개선이 가능한 영역인지 확인해줘"

"system : 인공지능을 활용했을 때, 회귀, 분류 모델 중 어떤 모델이 적합한지 확인해줘"

"system : 개선이 필요한 영역을 한 줄로 정리해서 요약해줘"

"user : 인공지능을 적용하여여 개선이 필요한 영역은 {input}이야"
===============

"system : {요약} 영역에 대해 인공지능을 활용하고 싶어"

"system : 인공지능이 예측하려는 Target이 무엇인지 확인해줘"

"system : 인공지능의 목적을 한 줄로 정리해서 요약해줘"

"user : 인공지능의 목적은 {input}이야"

=============

"AI의 주제는 {요약}이야"

"입력된 데이터를 feature와 label 두 종류로 구분해줘"

"python script를 사용하여 dataframe을 생성해줘"

"dataframe 생성 시, column의 명칭은 입력된 feature와 Label 명칭으로 사용해줘"

"dataframe의 value값들은 랜덤으로 50개만 생성해줘"

"value 값 생성 시, feature의 type에 맞게 생성해줘"

"생성한 데이터프레임을 csv 파일로 저장해줘"

"생성한 dataframe의 저장 경로는 /ml/algorithmlabs.csv 폴더야"

"저장하는 데이터의 이름은 'test_csv'야"

"위 단계들을 차근차근 생각해줘"

"user : 예시 데이터 세트는 {input}이야. dataframe을 생성하고 저장해줘"

# review

"지금부터 python code를 리뷰할거야"

"차근차근 생각하고 효율적으로 코드를 구축해줘"

"생성한 python code가 실행하는데 문제가 없는지 체크해줘"

"실행하는데 문제가 발생하지 않을 경우 python script를 그대로 둬"

"실행하는데 문제가 발생할 경우 python script 코드를 수정해줘"

"user : 생성한 {answer}를 리뷰해주고 python script를 다시 전달해줘"

=================================

"개선이 필요한 프로세스는 {요약1}이야"

"인공지능의 목적은 {요약2}이야"

"현재 보유한 데이터프레임은 {python script}이야"

"생성한 데이터프레임이 인공지능 회귀, 분류 모델 중 무엇이 적합한지 생각해줘"

"인공지능 회귀 모델이 적합할 경우 사용가능한 plot들을 3개만 나열해줘"

"인공지능 분류 모델이 적합할 경우 사용가능한 plot들을 3개만 나열해줘"

"데이터프레임의 feature type들을 참고하여 사용할 수 있는 plot들을 3개만 나열해줘"

"plot들을 나열할 시 반드시 {visualize plot}에 있는 plot들만 추천해줘"

"user : 데이터프레임을 참고하여 인공지능 모델에 필요한 시각화 plot들을 알려줘"

# review

"{plot}들에 대해 리뷰할게"

"{plot}이 {visualize plot} 목록 내에 모두 존재하면 그대로 결과를 나열해줘"

" 만약 목록 내에 존재하지 않는다면 {visualize plot}을 기반으로 다시 나열해줘"

"현재 보유한 데이터프레임은 {python script}이야"

"생성한 데이터프레임이 인공지능 회귀, 분류 모델 중 무엇이 적합한지 생각해줘"

"인공지능 회귀 모델이 적합할 경우 사용가능한 plot들을 3개만 나열해줘"

"인공지능 분류 모델이 적합할 경우 사용가능한 plot들을 3개만 나열해줘"

"데이터프레임의 feature type들을 참고하여 사용할 수 있는 plot들을 3개만 나열해줘"

"plot들을 나열할 시 반드시 {visualize plot}에 있는 plot들만 추천해줘"

"user : 데이터프레임을 참고하여 인공지능 모델에 필요한 시각화 plot들을 알려줘"